In [1]:
from sbi.inference import MNLE, simulate_for_sbi
import numpy as np
import dill
from multiprocessing import cpu_count
import matplotlib.pyplot as plt
import sys

In [2]:
def simulate_PA(params):
    v,a = params
    dt = 1e-3; dB = np.sqrt(dt)
    DV = 0; t = 0
    while True:
        DV += v*dt + np.random.normal(0, dB)
        t += dt

        if DV >= a:
            return t, 1

In [ ]:
from torch.distributions import Uniform
import torch
v_distr = Uniform(torch.tensor([0.1]), torch.tensor([3]))    # decision boundary
a_distr = Uniform(torch.tensor([0.1]), torch.tensor([3]))    # decision boundary
from sbi.utils import MultipleIndependent

prior = MultipleIndependent(
    [
        v_distr,
        a_distr
    ]
)

In [5]:
from sbi.utils.user_input_checks import process_simulator, process_prior, check_sbi_inputs
prior, num_parameters, prior_returns_numpy = process_prior(prior)
simulator = process_simulator(simulate_PA, prior, prior_returns_numpy)


In [6]:
from sbi.inference import simulate_for_sbi
# theta, x = simulate_for_sbi_truncated(simulator=simulator_2D_x, proposal=proposal, num_simulations=num_simulations, num_workers=n_cores-1)
theta, x = simulate_for_sbi(simulator, prior, num_simulations=1000, num_workers=32)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [18]:
trainer = MNLE()    
estimator = trainer.append_simulations(theta=theta,x=x).train(training_batch_size=1000)

RuntimeError: index 1 is out of bounds for dimension 2 with size 1